<a href="https://colab.research.google.com/github/sanisammani/Chatbot-/blob/main/Textscam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (update path if needed)
file_path = '/content/scamdata.csv'  # Update this if the file path changes
try:
    scam_data = pd.read_csv(file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"File not found at {file_path}. Please check the path.")

# Ensure necessary columns exist in the dataset
required_columns = ['input', 'output']
missing_columns = [col for col in required_columns if col not in scam_data.columns]
if missing_columns:
    raise ValueError(f"The dataset is missing required columns: {missing_columns}")

# Preprocess the data
scam_data['input'] = scam_data['input'].str.replace("Classify this message:", "", regex=True).str.strip().str.lower()

# Validate preprocessing
if scam_data['input'].isnull().any() or scam_data['output'].isnull().any():
    raise ValueError("Null values found in 'input' or 'output' columns after preprocessing.")

# Split the data into training, validation, and test sets
try:
    train_data, temp_data = train_test_split(
        scam_data, test_size=0.2, random_state=42, stratify=scam_data['output']
    )
    val_data, test_data = train_test_split(
        temp_data, test_size=0.5, random_state=42, stratify=temp_data['output']
    )
except ValueError as e:
    raise ValueError(f"Error during train-test split: {e}")

# Output summary of splits
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")



Training set size: 4457
Validation set size: 557
Test set size: 558


In [10]:
from transformers import T5Tokenizer

# Tokenize data (Ensure these steps have been executed prior)
def tokenize_data(inputs, targets, tokenizer, max_length=512):
    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    target_encodings = tokenizer(targets, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return encodings.input_ids, encodings.attention_mask, target_encodings.input_ids

train_inputs, train_targets = format_data_for_t5(train_data)
train_input_ids, train_attention_masks, train_labels = tokenize_data(train_inputs, train_targets, tokenizer)


In [11]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from transformers import AdamW

# Create DataLoader
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Loss: 0.7169264756092187
Epoch 2, Loss: 0.03167866690316543
Epoch 3, Loss: 0.02116043094864702
